In [ ]:
import requests
import pandas as pd
import json
from uuid import uuid4


In [ ]:
accident = pd.read_csv('Road Accident Data.csv')
sample_size = int(len(accident)* 0.01)
accident = accident.sample(n=sample_size, random_state=42)

try:
    accident['Accident Date'] = pd.to_datetime(accident['Accident Date'], errors='coerce').dt.strftime('%Y-%m-%d')
except Exception as e:
    print(f"Erreur lors de la conversion des dates : {e}")


In [13]:
import time

url = "https://archive-api.open-meteo.com/v1/era5"

weather_data = []

for _, row in accident.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    date = row['Accident Date']
    index = row["Accident_Index"]


    # Paramètres pour l'API Open-Meteo
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,shortwave_radiation_sum,sunrise,sunset",
        "timezone": "auto"
    }

    try:
        response = requests.get(url, params=params, timeout = 10)
        data = response.json()

        # Vérification si les données "daily" sont disponibles
        if "daily" in data:
            daily_data = data["daily"]

            # Extraction et calcul des données nécessaires
            max_temp = daily_data["temperature_2m_max"][0] if "temperature_2m_max" in daily_data else None
            min_temp = daily_data["temperature_2m_min"][0] if "temperature_2m_min" in daily_data else None
            total_precipitation = daily_data["precipitation_sum"][0] if "precipitation_sum" in daily_data else None
            wind_speed = daily_data["windspeed_10m_max"][0] if "windspeed_10m_max" in daily_data else None
            solar_radiation = daily_data["shortwave_radiation_sum"][0] if "shortwave_radiation_sum" in daily_data else None

            # Calcul de la durée totale de jour en heures
            sunrise = pd.to_datetime(daily_data["sunrise"][0]) if "sunrise" in daily_data else None
            sunset = pd.to_datetime(daily_data["sunset"][0]) if "sunset" in daily_data else None
            day_length = None
            if sunrise and sunset:
                day_length = (sunset - sunrise).seconds / 3600  # Convertir en heures

            # Ajouter les données collectées dans la liste
            weather_data.append({
                "AccidentIndex": index,
                "Date": date,
                "MaxTemperature": max_temp,
                "MinTemperature": min_temp,
                "TotalPrecipitation": total_precipitation,
                "MaxWindSpeed": wind_speed,
                "SolarRadiation": solar_radiation,
                "DayLengthHours": day_length
            })

           
           
    except Exception as e:
        print(f"Erreur pour l'index {index}: {e}")
        time.sleep(1)
# Sauvegarder les données météo dans un fichier JSON
with open('weather_data2.json', 'w') as json_file:
    json.dump(weather_data, json_file, indent=4)

print("Données météorologiques ajoutées avec succès !")

Erreur pour l'index 343ef0fe-c0c8-4b23-b7a2-83f15d8dd080: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur pour l'index 200904EA09135: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur pour l'index f93c8104-94be-4373-bf6e-9e87912105ae: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur pour l'index 2009559D01234: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur pour l'index 200922E904889: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Erreur pour l'index 200901LX51038: ('Connection aborted.', ConnectionResetError(10054, 'Une connexion ex

Convertir au format parquet

In [14]:
try:
    # Convertir la liste des dictionnaires en DataFrame
    weather_df = pd.DataFrame(weather_data)

    # Sauvegarder au format Parquet
    weather_df.to_parquet('weather_data.parquet', engine='pyarrow', index=False)

    print("Données météorologiques ajoutées avec succès au format Parquet !")
except Exception as e:
    print(f"Erreur lors de la conversion en Parquet : {e}")

Données météorologiques ajoutées avec succès au format Parquet !
